## Código basado en el siguiente artículo:

https://medium.com/towards-artificial-intelligence/whatsapp-group-chat-analysis-using-python-and-plotly-89bade2bc382

In [ ]:
# Importamos las librerías que vamos a usar
import re, pandas as pd, datetime, numpy as np, emoji, plotly.express as px
from collections import Counter

### Generamos un DataFrame de pandas a través del chat exportado:

In [ ]:
def getDfFromChat(chatFile):
    data = extractData(chatFile)
    return pd.DataFrame(data, columns=['Date', 'Time', 'Author', 'Message'])
def extractData(chatFile):
    data = []
    for msg in open(chatFile, encoding="utf8"):
        d = getDataFromLine(msg)
        if len(d) > 0:
            if len(d[0]) == 4:
                data.append(d[0])
    return data

def getDataFromLine(line):
    regexDate = "^([0-9]+\/[0-9]+\/[0-9][0-9]) "
    regexHour = "([0-9]+:[0-9][0-9]) "
    regexAuthor = "- (.*?)\: "
    regexMsg = "(.*)"
    regexAll = regexDate + regexHour + regexAuthor + regexMsg 
    return re.findall(regexAll, line)

In [ ]:
df = getDfFromChat("Chat.txt")

In [ ]:
df

### Dividimos el DataFrame en dos: 
* Mensajes multimedia
* Mensajes de texto 

In [ ]:
df_Multimedia = df[df['Message'] == '<Multimedia omitido>']
dfMensajes = df.drop(df_Multimedia.index)

### Enriquecemos el chat con:

* Emojis usados
* URLs compartidas
* Longitud de mensajes
* Numero de palabras

In [ ]:
def enriquecerChat(dfMensajes, df_Multimedia):
    # Emojis
    dfMensajes["emoji"] = dfMensajes["Message"].apply(split_count)
    # URLs
    URLPATTERN = r'(https?://\S+)'
    dfMensajes['urlcount'] = dfMensajes.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
    # Longitud de los mensajes
    dfMensajes['Letter_Count'] = dfMensajes['Message'].apply(lambda s : len(s))
    # Numero de palabres en un mensaje
    dfMensajes['Word_Count'] = dfMensajes['Message'].apply(lambda s : len(s.split(' ')))
def split_count(s):
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI)


In [ ]:
enriquecerChat(dfMensajes, df_Multimedia)

In [ ]:
dfMensajes

### Estadísticas generales

In [ ]:
l = dfMensajes.Author.unique()

for i in range(len(l)):
  req_df= dfMensajes[dfMensajes["Author"] == l[i]]
  print(f'Estadisticas de {l[i]}:')
  print()
  print('\tMensajes enviados:', req_df.shape[0])
  words_per_message = (np.sum(req_df['Word_Count']))/req_df.shape[0]
  print('\tPalabras por mensaje:', words_per_message)
  media = df_Multimedia[df_Multimedia['Author'] == l[i]].shape[0]
  print('\tMensajes multimedia enviados:', media)
  emojis = sum(req_df['emoji'].str.len())
  print('\tEmojis enviados:', emojis)
  links = sum(req_df["urlcount"])   
  print('\tLinks enviados:', links)
  print()

# Estadísticas de Emojis

### Tabla de emojis usados

In [ ]:

total_emojis_list = list([a for b in dfMensajes.emoji for a in b])
emoji_dict = dict(Counter(total_emojis_list))
emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)

emoji_df = pd.DataFrame(emoji_dict, columns=['emoji', 'count'])


In [ ]:
emoji_df

## Graficos circulares

### Total

In [ ]:

fig = px.pie(emoji_df, values='count', names='emoji',
             title='Emoji Distribution')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### Por autor

In [ ]:
l = dfMensajes.Author.unique()
for i in range(len(l)):
  dummy_df = dfMensajes[dfMensajes['Author'] == l[i]]
  total_emojis_list = list([a for b in dummy_df.emoji for a in b])
  emoji_dict = dict(Counter(total_emojis_list))
  emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)
  print('Emoji Distribution for', l[i])
  author_emoji_df = pd.DataFrame(emoji_dict, columns=['emoji', 'count'])
  fig = px.pie(author_emoji_df, values='count', names='emoji')
  fig.update_traces(textposition='inside', textinfo='percent+label')
  fig.show()

## Total de palabras

In [ ]:
text = " ".join(review for review in dfMensajes.Message)
print ("There are {} words in all the messages.".format(len(text)))

## Palabras más usadas

Para el siguiente necesitaremos descargar el paquete de Stop-words (Palabras vacías) en español. Estas son artículos, pronombres, preposiciones que no nos interesa graficar.

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud,STOPWORDS
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

### Usamos el español, a su vez podemos agregar otras palabras que no queramos visualizar

In [ ]:
stopwords_spanish = set(stopwords.words('spanish'))
# stopwords = set(stopwords.words('spanish'))

stopwords_spanish.update(["jajajaja", "jajaja"])

In [ ]:
text = " ".join(review for review in dfMensajes.Message)

wordcloud = WordCloud(stopwords=stopwords_spanish, background_color="white").generate(text)
# Display the generated image:
# the matplotlib way:

plt.figure( figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## Grafico mensajes-día a lo largo del tiempo

Cada columna es un mensaje, añadiendo el valor de la siguiente celda al agruparlas por fecha nos dará el número de mensajes por día 

In [ ]:
dfMensajes["MessageCount"] = 1

In [ ]:
# Agrupamos por fecha, Esto convierte a la fecha en el índice.
date_df = dfMensajes.groupby("Date").sum()
# Hacemos que la fecha pase a ser otra columna
date_df.reset_index(inplace=True)
# Transformamos la columna fecha para que pandas la entienda como tal y la podamos ordenar
date_df['Date'] =pd.to_datetime(date_df.Date, format= "%d/%m/%y")
# Ordenamos por fecha
date_df = date_df.sort_values(by='Date')
# Reiniciamos el índice. De no hacerlo el grafico uniría los puntos por indice, los cuales no son los mismos que le fecha y saldría un garabato. Al reiniciar el índice se crea una columna Index, la eliminamos.
date_df.reset_index().drop(["index"], axis=1)


### Graficamos

In [ ]:
fig = px.line(date_df, x="Date", y="MessageCount", title='Número de mensajes a lo largo del tiempo')
fig.update_xaxes(nticks=20)
fig.show()

### Días con más mensajes

In [ ]:
dfMensajes['Date'].value_counts().head(10).plot.barh()
plt.xlabel('Número de mensajes')
plt.ylabel('Fecha')

## Número de mensajes repartidos por días

In [ ]:
dfMensajes["Date"] = pd.to_datetime(dfMensajes.Date, format= "%d/%m/%y")

def dayofweek(i):
  l = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
  return l[i]
day_df=pd.DataFrame(dfMensajes["Message"])
day_df['day_of_date'] = dfMensajes['Date'].dt.weekday
day_df['day_of_date'] = day_df["day_of_date"].apply(dayofweek)
day_df["messagecount"] = 1
day = day_df.groupby("day_of_date").sum()
day.reset_index(inplace=True)

fig = px.line_polar(day, r='messagecount', theta='day_of_date', line_close=True)
fig.update_traces(fill='toself')
rangoMensajes = day["messagecount"].sort_values(ascending=False).head(1).iloc[0] + 100
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,      
      range=[0,rangoMensajes]
    )),
  showlegend=False
)
fig.show()

### Momentos del día con más mensajes

In [ ]:
dfMensajes['Time'].value_counts().head(10).plot.barh()
plt.xlabel('Number of messages')
plt.ylabel('Time')